In [ ]:
from lsst.rsp import get_tap_service
import sqlite3
import pandas as pd

In [ ]:
service = get_tap_service("ssotap")

In [ ]:
db_fname = "adler_demo_testing_database.db"

In [ ]:
cnx = sqlite3.connect(db_fname)

In [ ]:
# ssoid = 8268570668335894776
ssoid_list = (8268570668335894776, 2150553186630, 3369984299447, 5992863104062, 6098332225018)

test_id = 6098332225018
fake_id = 6098332225018000

In [ ]:
filter_name = "r"

In [ ]:
# diasource_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.diaSource
#             WHERE
#                 ssObjectId = {ssoid}
#             """
diasource_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.diaSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
diatable = service.search(diasource_sql_query).to_table().to_pandas()

In [ ]:
# sssource_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.SSSource
#             WHERE
#                 ssObjectId = {ssoid}
#             """
sssource_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.SSSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
sssource_table = service.search(sssource_sql_query).to_table().to_pandas()

In [ ]:
sssource_table.columns

In [ ]:
# ssobject_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.SSObject
#             WHERE
#                 ssObjectId = {ssoid}
#             """
ssobject_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.SSObject
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
ssobject_table = service.search(ssobject_sql_query).to_table().to_pandas()

In [ ]:
ssobject_table.columns

In [ ]:
ssobject_table["r_Ndata"]

In [ ]:
# mpcorb_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.MPCORB
#             WHERE
#                 ssObjectId = {ssoid}
#             """
mpcorb_sql_query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.MPCORB
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
mpcorb_table = service.search(mpcorb_sql_query).to_table().to_pandas()

In [ ]:
mpcorb_table

In [ ]:
# add a new fake object

In [ ]:
diatable

In [ ]:
diatable[(diatable["ssObjectId"] == test_id)].value_counts("diaSourceId")

In [ ]:
diatable[(diatable["ssObjectId"] == test_id) & (diatable["band"] == "r")].value_counts("diaSourceId")

In [ ]:
# df_dict = {"dia":diatable,
#            "sss":sssource_table,
#            "sso":ssobject_table,
#            "mpc":mpcorb_table}

In [ ]:
# for x in df_dict:

#     _df = df_dict[x].copy()[df_dict[x]["ssObjectId"]==test_id]
#     _df["ssObjectId"] = fake_id
#     df_dict[x] = pd.concat([df_dict[x],_df],axis = 0).reset_index(drop=True)
#     print(len(_df),len(df_dict[x]))

In [ ]:
# diatable = df_dict["dia"]
# sssource_table = df_dict["sss"]
# ssobject_table = df_dict["sso"]
# mpcorb_table = df_dict["mpc"]

In [ ]:
# _df = diatable.copy()[diatable["ssObjectId"]==test_id]
# _df["ssObjectId"] = fake_id
# diatable = pd.concat([diatable,_df],axis = 0).reset_index(drop=True)
# # print(len(_df),len(df_dict[x]))
# _df

In [ ]:
# # add an outburst
# t0 = 63100
# t1 = 63600
# mag_shift = -1.5

# mask = ((diatable["ssObjectId"] == fake_id) &
#         (diatable["midPointMjdTai"] > t0) &
#         (diatable["midPointMjdTai"] < t1))

# diatable.loc[mask,"mag"] += mag_shift

In [ ]:
diatable

In [ ]:
# diatable.to_sql("diaSource", con=cnx, if_exists="append", index=False)
diatable.to_sql("diaSource", con=cnx, if_exists="replace", index=False)

In [ ]:
# sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)
sssource_table.to_sql("ssSource", con=cnx, if_exists="replace", index=False)

In [ ]:
# ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)
ssobject_table.to_sql("ssObject", con=cnx, if_exists="replace", index=False)

In [ ]:
# mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)
mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="replace", index=False)

In [ ]:
_diatable = diatable.copy()[diatable["ssObjectId"] == test_id]
_diatable["ssObjectId"] = fake_id
_diatable["diaSourceId"] = _diatable["diaSourceId"] * 1000

# add an outburst
t0 = 63100
t1 = 63600
mag_shift = -1.5

mask = (
    (_diatable["ssObjectId"] == fake_id)
    & (_diatable["midPointMjdTai"] > t0)
    & (_diatable["midPointMjdTai"] < t1)
)

_diatable.loc[mask, "mag"] += mag_shift

_diatable.to_sql("diaSource", con=cnx, if_exists="append", index=False)

In [ ]:
_diatable[diatable["band"] == "r"]

In [ ]:
_sssource_table = sssource_table.copy()[sssource_table["ssObjectId"] == test_id]
_sssource_table["ssObjectId"] = fake_id
_sssource_table["diaSourceId"] = _sssource_table["diaSourceId"] * 1000
_sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)

_ssobject_table = ssobject_table.copy()[ssobject_table["ssObjectId"] == test_id]
_ssobject_table["ssObjectId"] = fake_id
_ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)

_mpcorb_table = mpcorb_table.copy()[mpcorb_table["ssObjectId"] == test_id]
_mpcorb_table["ssObjectId"] = fake_id
_mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)

In [ ]:
cnx.close()

Testing everything went correctly...

In [ ]:
cnx = sqlite3.connect(db_fname)

In [ ]:
# example_query = f"""
#                 SELECT
#                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
#                     topocentricDist, heliocentricDist
#                 FROM
#                     ssObject
#                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
#                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
#                 WHERE
#                     ssObject.ssObjectId = {ssoid}
#                 """
# example_query = f"""
#                 SELECT
#                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
#                     topocentricDist, heliocentricDist
#                 FROM
#                     ssObject
#                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
#                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
#                 WHERE
#                     ssObject.ssObjectId in {ssoid_list}
#                 """

example_query = f"""
                SELECT
                    ssObject.ssObjectId, ssSource.diaSourceId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
                    topocentricDist, heliocentricDist
                FROM
                    ssObject
                    JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
                    JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
                WHERE
                    ssObject.ssObjectId = {fake_id} and band = 'r'
                """

In [ ]:
pd.read_sql_query(example_query, cnx)

In [ ]:
cur = cnx.cursor()

In [ ]:
res = cur.execute("SELECT * FROM sqlite_schema")

In [ ]:
res.fetchall()